Evaluación II

Evaluación de contenidos 2 Módulo 2



In [1]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

Bienvenidas a una nueva evaluación del Bootcamp de Análisis de Datos de Adalab.
Como en evaluaciones anteriores dispondréis de 2 tardes (aproximadamente 8 horas para la resolución de todos los ejercicios propuestos).


Estos ejercicios se tendrán que defender en el día de la evaluación de forma individual con vuestra tutora docente en una simulación de entrevista técnica donde tendréis de 20 minutos para resolver o explicar los ejercicios propuestos entregados.


En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.


Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.

Es importante tener en cuenta que este proyecto requerirá un conocimiento profundo de herramientas y técnicas de análisis de datos, así como habilidades en programación y manejo de APIs. 
También es importante tener una comprensión sólida de la estructura y organización de los datos, ya que esto nos permitirá hacer preguntas importantes y obtener respuestas significativas a partir de los datos.


En resumen, esta prueba técnica ofrece una excelente oportunidad para demostrar habilidades y conocimientos en análisis de datos y programación, mientras se trabaja en un proyecto real y relevante para una empresa. Al finalizar del proyecto, esperamos obtener información valiosa que ayudará a la empresa a tomar decisiones más informadas sobre las universidades en los tres países objetivo.

NOTA DE LA ALUMNA:
*Los comentarios sobre el código se incluyen en cursiva durante el ejercicio*

1.
Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:

*Preparamos una constante que usaremos en diferentes llamadas*

In [2]:
API_URL = "http://universities.hipolabs.com/search?country="

In [3]:
pd.options.display.max_columns = None

In [4]:
geolocator = Nominatim(user_agent='geo')

*Usamos la API para obtener los datos de Canadá, Estados Unidos y Argentina*

In [5]:

response = requests.get(url= API_URL + 'Canada')

df_uni = pd.json_normalize(response.json())

response = requests.get(url= API_URL + 'United States')

df_uni = pd.concat([df_uni, pd.json_normalize(response.json())] )

response = requests.get(url= API_URL + 'Argentina')

df_uni = pd.concat([df_uni, pd.json_normalize(response.json())] )




2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:

> Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.


*Creamos una dictionary comprehension que contenga los cambios a realizar a las columnas y la igualamos a una variable.* 

*Usamos el método rename sobre el dataframe, igualando columns a la variable que contiene el diccionario con los cambios. Debemos incluir el inplace = True para que nos sobreescriba el dataframe.*

In [6]:
nuevas_columnas = {col: col.replace("-", "_").lower() for col in df_uni.columns}
df_uni.rename(columns = nuevas_columnas, inplace = True)



> La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

*Usamos el método drop sobre la columna domains, especificando el axis =1, las columnas.*

In [7]:
df_uni = df_uni.drop('domains', axis=1)


3.Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. 

Esto es poco práctico y puede llegar a no tener sentido. El objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. Tenéis la documentación de este método.
Al final os quedará una tabla similar a la siguiente:

In [8]:
df_uni.explode('web_pages')

,state_province,name,country,web_pages,alpha_two_code
0,Quebec,Cégep de Saint-Jérôme,Canada,https://www.cstj.qc.ca,CA
0,Quebec,Cégep de Saint-Jérôme,Canada,https://ccmt.cstj.qc.ca,CA
0,Quebec,Cégep de Saint-Jérôme,Canada,https://ccml.cstj.qc.ca,CA
1,Ontario,Lambton College,Canada,https://www.lambtoncollege.ca,CA
2,Nova Scotia,Acadia University,Canada,http://www.acadiau.ca/,CA
...,...,...,...,...,...
82,Buenos Aires,Universidad Nacional de Tres de Febrero,Argentina,http://www.untref.edu.ar/,AR
83,Ciudad Autónoma de Buenos Aires,Universidad Torcuato di Tella,Argentina,http://www.utdt.edu/,AR
84,Ciudad Autónoma de Buenos Aires,Universidad Tecnológica Nacional,Argentina,http://www.utn.edu.ar/,AR
85,Ciudad Autónoma de Buenos Aires,Universidad Abierta Interamericana,Argentina,http://www.vaneduc.edu.ar/uai/,AR


4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

*Usando el método drop_duplicates eliminaremos los duplicados existentes en la columna 'name'.*

In [13]:
df_uni.drop_duplicates(subset = ['name'], inplace = True)

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [14]:
df_uni['state_province'].isnull().sum()

0

In [15]:
df_uni['state_province'].fillna(np.NaN, inplace=True)

6.Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknown". 

No nos piden ningún método especifico, así que podremos usar el método que queramos

*Usamos el método .fillna() sobre la columna 'state_province' con inplace = True para sobre escribir*

In [11]:
df_uni['state_province'].fillna('Unknown', inplace=True)

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso,  la documentación. Para desarrollar este ejercicio deberéis:




> Sacar los valores únicos de la columna state_province.


In [16]:
df_uni['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknown',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY', 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires',
       'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza', 'Santa Fé',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán'],
      dtype=object)

In [27]:
new_states = {
    'NV':'Nevada', 
    'TX':'Texas',
    'IN':'Indianapolis',
    'CA':'California',
    'VA':'Virginia',
    'NY':'New York',
    'MI':'Michigan',
    'GA':'Georgia',
    'ND':'North Dakota',
    'New York, NY': 'New York',
    'Ciudad Autónoma de Buenos Aires': 'Buenos Aires'}

df_uni['state_province'].replace(new_states, inplace = True)


In [30]:
df_uni['state_province'].unique()

array(['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan', 'Unknown',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Nevada', 'Iowa', 'Virginia', 'Texas', 'Colorado',
       'Indianapolis', 'California', 'South Carolina', 'Washington',
       'New York', 'North Dakota', 'Michigan', 'Ohio', 'Florida',
       'North Carolina', 'Georgia', 'Buenos Aires', 'Entre Ríos', 'Salta',
       'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)




Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:




> NV: reemplazalo por Nevada



>TX: reemplazalo por Texas


>IN: reemplazalo por Indianapolis


>CA: reemplazalo por California


>VA: reemplazalo por Virginia


>NY: reemplazalo por New York


>MI: reemplazalo por Michigan


>GA: reemplazalo por Georgia


>ND: reemplazalo por North Dakota




Otros valores que tenemos más formateados son y que deberemos reemplazar:

>New York, NY. Deberéis reemplazarlo por "New York".


> 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"




> Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.


In [34]:
states_list = df_uni['state_province'].unique().tolist()

states_list

['Quebec',
 'Ontario',
 'Nova Scotia',
 'British Columbia',
 'Alberta',
 'Manitoba',
 'New Brunswick',
 'Saskatchewan',
 'Unknown',
 'Newfoundland and Labrador',
 'Prince Edward Island',
 'Yukon',
 'Pennsylvania',
 'Nevada',
 'Iowa',
 'Virginia',
 'Texas',
 'Colorado',
 'Indianapolis',
 'California',
 'South Carolina',
 'Washington',
 'New York',
 'North Dakota',
 'Michigan',
 'Ohio',
 'Florida',
 'North Carolina',
 'Georgia',
 'Buenos Aires',
 'Entre Ríos',
 'Salta',
 'Córdoba',
 'Mendoza',
 'Santa Fé',
 'Santiago Del Estero',
 'Misiones',
 'Catamarca',
 'Formosa',
 'Jujuy',
 'La Rioja',
 'La Pampa',
 'San Juan',
 'San Luis',
 'Tucumán']

>Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.


In [35]:
pip install geopy

     |████████████████████████████████| 119 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 6.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [52]:
df_geo = pd.DataFrame(states_list, columns= ['state'])
df_geo.head()

,state
0,Quebec
1,Ontario
2,Nova Scotia
3,British Columbia
4,Alberta


https://stackoverflow.com/questions/23330654/update-a-dataframe-in-pandas-while-iterating-row-by-row

In [58]:
for i, row in df_geo.iterrows():
    location = geolocator.geocode(row['state'])
    df_geo.at[i, 'latitude'] = location.latitude
    df_geo.at[i, 'longitude'] = location.longitude

>Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

https://stackoverflow.com/questions/20375561/joining-pandas-dataframes-by-column-names

In [64]:
df_uni = df_uni.merge(df_geo, left_on = 'state_province', right_on = 'state')

In [65]:
df_uni = df_uni.drop('state', axis = 1)

8. Crea una BBDD en mysql que contenga las siguientes tablas:

*Este ejercicio se encuentra en una carpeta llamada anexo*

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

*Este ejercicio se presenta en un archivo extra llamado funciones*